In [ ]:
from sklearn.metrics.pairwise import pairwise_distances
import tensorflow as tf
import numpy as np
import time
import h5py
import sys

In [ ]:
def average_gradients(tower_grads):
    average_grads = []
    for grad_and_vars in zip(*tower_grads):
        # Note that each grad_and_vars looks like the following:
        #   ((grad0_gpu0, var0_gpu0), ... , (grad0_gpuN, var0_gpuN))
        grads = []
        for g, _ in grad_and_vars:
            # Add 0 dimension to the gradients to represent the tower.
            expanded_g = tf.expand_dims(g, 0)

            # Append on a 'tower' dimension which we will average over below.
            grads.append(expanded_g)

        # Average over the 'tower' dimension.
        grad = tf.concat(axis=0, values=grads)
        grad = tf.reduce_mean(grad, 0)

        # Keep in mind that the Variables are redundant because they are shared
        # across towers. So .. we will just return the first tower's pointer to
        # the Variable.
        v = grad_and_vars[0][1]
        grad_and_var = (grad, v)
        average_grads.append(grad_and_var)
    return average_grads

In [ ]:
def identity_block(input_tensor, kernel_size, filters, stage, block, is_training):
    with tf.variable_scope("stage-{}_block-{}".format(stage, block), reuse=tf.AUTO_REUSE) as scope:
        x = tf.layers.batch_normalization(input_tensor, training=is_training)
        x = tf.nn.relu(x)
        x = tf.layers.conv2d(x, filters[0], kernel_size[0], padding="same")

        x = tf.layers.batch_normalization(x, training=is_training)
        x = tf.nn.relu(x)
        x = tf.layers.conv2d(x, filters[1], kernel_size[1], padding="same")

        x = tf.concat([x, input_tensor], axis=-1)
    return x

def conv_block(input_tensor, kernel_size, filters, stage, block, is_training, strides=2):
    with tf.variable_scope("stage-{}_block-{}".format(stage, block), reuse=tf.AUTO_REUSE) as scope:
        x = tf.layers.batch_normalization(input_tensor, training=is_training)
        x = tf.nn.relu(x)
        x = tf.layers.conv2d(x, filters[0], kernel_size[0], strides=strides, padding="same")

        x = tf.layers.batch_normalization(x, training=is_training)
        x = tf.nn.relu(x)
        x = tf.layers.conv2d(x, filters[1], kernel_size[1], padding="same")

        shortcut = tf.layers.batch_normalization(input_tensor, training=is_training)
        shortcut = tf.nn.relu(shortcut)
        shortcut = tf.layers.conv2d(shortcut, filters[1], kernel_size[1], strides=strides, padding="same")

        x = tf.concat([x, shortcut], axis=-1)
    return x

def classifier(inputs, num_classes, num_embeddings, is_training):
    with tf.variable_scope("stage-1_block-a", reuse=tf.AUTO_REUSE) as scope:
        x = tf.layers.conv2d(inputs, 64, 7, strides=2, padding="same")
        x = tf.layers.batch_normalization(x, training=is_training)
        x = tf.nn.relu(x)
        x = tf.layers.max_pooling2d(x, 3, strides=2, padding="same")

    x = conv_block    (x, [1, 3], [64, 256], stage=2, block='a', is_training=is_training)
    x = identity_block(x, [1, 3], [64, 256], stage=2, block='b', is_training=is_training)

    x = conv_block    (x, [1, 3], [128, 512], stage=3, block='a', is_training=is_training)
    x = identity_block(x, [1, 3], [128, 512], stage=3, block='b', is_training=is_training)

    x = conv_block    (x, [1, 3], [256, 1024], stage=4, block='a', is_training=is_training)
    x = identity_block(x, [1, 3], [256, 1024], stage=4, block='b', is_training=is_training)

    x = conv_block    (x, [1, 3], [512, 2048], stage=5, block='a', is_training=is_training)
    x = identity_block(x, [1, 3], [512, 2048], stage=5, block='b', is_training=is_training)

    with tf.variable_scope("embeddings", reuse=tf.AUTO_REUSE) as scope:
        x = tf.layers.average_pooling2d(x, (x.get_shape()[-3], x.get_shape()[-2]), 1) #global average pooling
        x = tf.layers.flatten(x)
        embeddings = tf.layers.dense(x, num_embeddings, activation=None)
        
    with tf.variable_scope("logits", reuse=tf.AUTO_REUSE) as scope:
        pred_logits = tf.nn.relu(embeddings)
        pred_logits = tf.layers.dense(pred_logits, num_classes, activation=None)

    return embeddings, pred_logits

In [ ]:
def get_center_loss(features, labels, alpha, num_classes):
    
    len_features = features.get_shape()[1] 
    with tf.variable_scope("central_loss", reuse=tf.AUTO_REUSE) as scope:
        centers = tf.get_variable('centers', [num_classes, len_features], dtype=tf.float32,
            initializer=tf.constant_initializer(0), trainable=False)
        
    labels = tf.reshape(labels, [-1])

    centers_batch = tf.gather(centers, labels)
    loss = tf.nn.l2_loss(features - centers_batch)

    unique_label, unique_idx, unique_count = tf.unique_with_counts(labels)
    appear_times = tf.gather(unique_count, unique_idx)
    appear_times = tf.reshape(appear_times, [-1, 1])

    diff = centers_batch - features
    diff = diff / tf.cast((1 + appear_times), tf.float32)
    diff = alpha * diff

    centers_update_op = tf.scatter_sub(centers, labels, diff)
    tf.add_to_collection(tf.GraphKeys.UPDATE_OPS, centers_update_op)

    return loss, centers

In [ ]:
def unison_shuffled_copies(a, b):
    p = np.random.permutation(a.shape[0])
    return a[p], b[p]

def get_batch(data_x, data_y, batch_size):
    while True:
        data_x, data_y = unison_shuffled_copies(data_x, data_y)
        for index in range(0, data_x.shape[0], batch_size):
            x, y = data_x[index:index+batch_size], data_y[index:index+batch_size]
            if x.shape[0] == batch_size:
                yield x, y

In [ ]:
data_file = '/home/kjakkala/neuralwave/data/CSI_preprocessed_35.h5'
intruder_dir="/home/kjakkala/neuralwave/data/CSI_preprocessed_Intruder.h5"
tensorboard_path = "/home/kjakkala/neuralwave/data/logs/central_loss_135_99/"
weights_path =  "/home/kjakkala/neuralwave/data/weights/central_loss_135_99/central_loss_model.ckpt"
num_embeddings = 64
num_classes = 35
save_step = 10
decay_rate = 0.99
batch_size = 32
num_gpus = 4
alpha = 0.5
ratio = 0.5
epochs = 200
lr = 1e-4

hf = h5py.File(data_file, 'r')
X_train = np.expand_dims(hf.get('X_train'), axis=-1)[:, :, ::2]
X_test = np.expand_dims(hf.get('X_test'), axis=-1)[:, :, ::2]
y_train = np.eye(num_classes)[hf.get('y_train')]
y_test = np.eye(num_classes)[hf.get('y_test')]

y_train_n = np.array(hf.get('y_train'))
y_test_n = np.array(hf.get('y_test'))

hf.close()

hf = h5py.File(intruder_dir, 'r')
X_data = np.expand_dims(hf.get('X_data'), axis=-1)[:, :, ::2]
y_data = np.ones(X_data.shape[0])*-1
hf.close()

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_data.shape, y_data.shape, "\n")

train_steps = X_train.shape[0]//(batch_size*num_gpus)
test_steps = X_test.shape[0]//(batch_size*num_gpus)
train_data = get_batch(X_train, y_train, batch_size*num_gpus)
test_data = get_batch(X_test, y_test, batch_size*num_gpus)
rows, cols, channels = X_train.shape[1:]

In [ ]:
tf.reset_default_graph()

with tf.device('/cpu:0'):
    with tf.variable_scope("Inputs") as scope:
        X = tf.placeholder(tf.float32, [None, rows, cols, channels])
        Y = tf.placeholder(tf.float32, [None, num_classes])
        
    is_training = tf.placeholder(tf.bool)

    global_step = tf.train.get_or_create_global_step()

    learning_rate = tf.train.exponential_decay(lr,
                                               global_step,
                                               train_steps,
                                               decay_rate,
                                               staircase=True)
    tf.summary.scalar("learning_rate", learning_rate)       

    opt = tf.train.AdamOptimizer(learning_rate=learning_rate)      

    # Calculate the gradients for each model tower.
    tower_grads = []
    center_losses = []
    softmax_losses = []
    combined_losses = []
    accuracies = []
    for i in range(num_gpus):
        with tf.device("/gpu:{}".format(i)):
            with tf.name_scope("resnet_{}".format(i)) as scope:
                _x = X[i * batch_size: (i+1) * batch_size]
                _y = Y[i * batch_size: (i+1) * batch_size]

                embeddings, pred_logits = classifier(_x, num_classes, num_embeddings, is_training)

                center_loss, centers = get_center_loss(embeddings, tf.argmax(_y, axis=1), alpha, num_classes)
                softmax_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=_y, logits=pred_logits))
                combined_loss = softmax_loss + ratio * center_loss
                grads = opt.compute_gradients(combined_loss) 
                
                accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(pred_logits, 1), tf.argmax(_y, 1)), tf.float32))

                center_losses.append(center_loss)
                softmax_losses.append(softmax_loss)
                combined_losses.append(combined_loss)
                accuracies.append(accuracy)
                tower_grads.append(grads)

    grads = average_gradients(tower_grads)    
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        apply_gradient_op = opt.apply_gradients(grads, global_step)
        
    avg_center_loss = tf.reduce_mean(center_losses)
    avg_softmax_loss = tf.reduce_mean(softmax_losses)
    avg_combined_loss = tf.reduce_mean(combined_losses)
    avg_accuracies = tf.reduce_mean(accuracies)

    tf.summary.scalar("avg_center_loss", avg_center_loss)      
    tf.summary.scalar("avg_softmax_loss", avg_softmax_loss)      
    tf.summary.scalar("avg_combined_loss", avg_combined_loss) 
    tf.summary.scalar("avg_accuracies", avg_accuracies) 
    
    # Initializing the variables
    init = tf.global_variables_initializer()
    merged = tf.summary.merge_all()
    saver = tf.train.Saver(max_to_keep=5)

In [ ]:
# Launch the graph
with tf.Session(config=tf.ConfigProto(allow_soft_placement = True)) as sess:
    train_writer = tf.summary.FileWriter(tensorboard_path+"train", sess.graph)
    test_writer = tf.summary.FileWriter(tensorboard_path+"test", sess.graph)
    sess.run(init)

    for epoch in range(1, epochs + 1):
        for step in range(1, train_steps + 1):
            batch_x, batch_y = next(train_data)
            _, summary, curr_step = sess.run([apply_gradient_op, merged, global_step], feed_dict={X: batch_x, Y: batch_y, is_training: True})
            train_writer.add_summary(summary, curr_step)
        
        for step in range(1, test_steps+1):
            batch_x, batch_y = next(test_data)
            summary = sess.run(merged, feed_dict={X: batch_x, Y: batch_y, is_training: False})
            test_writer.add_summary(summary, (epoch*test_steps)+step)
            
        if (epoch % save_step == 0):
            saver.save(sess, weights_path, global_step=curr_step)
            
        sys.stdout.write("Current Epoch: {}\r".format(epoch))
        sys.stdout.flush()
            
    saver.save(sess, weights_path, global_step=curr_step)

In [ ]:
def fit(x, iters=1000, eps=1e-6):
    """
    Fits a 2-parameter Weibull distribution to the given data using maximum-likelihood estimation.
    :param x: 1d-ndarray of samples from an (unknown) distribution. Each value must satisfy x > 0.
    :param iters: Maximum number of iterations
    :param eps: Stopping criterion. Fit is stopped ff the change within two iterations is smaller than eps.
    :return: Tuple (Shape, Scale) which can be (NaN, NaN) if a fit is impossible.
        Impossible fits may be due to 0-values in x.
    """
    # fit k via MLE
    ln_x = np.log(x+eps)
    k = 1.
    k_t_1 = k

    for t in range(iters):
        x_k = x ** k
        x_k_ln_x = x_k * ln_x
        ff = np.sum(x_k_ln_x)
        fg = np.sum(x_k)
        f = ff / fg - np.mean(ln_x) - (1. / k)

        # Calculate second derivative d^2f/dk^2
        ff_prime = np.sum(x_k_ln_x * ln_x)
        fg_prime = ff
        f_prime = (ff_prime/fg - (ff/fg * fg_prime/fg)) + (1. / (k*k))

        # Newton-Raphson method k = k - f(k;x)/f'(k;x)
        k -= f/f_prime

        if np.isnan(f):
            return np.nan, np.nan
        if abs(k - k_t_1) < eps:
            break

        k_t_1 = k

    lam = np.mean(x ** k) ** (1.0 / k)

    return k, lam


def psi_i_dist(dist, k_i, lambda_i):
    """
    Gives the probability of sample inclusion
    :param dist: Numpy vector of distances between samples
    :param lambda_i: Scale of the Weibull fitting
    :param k_i: Shape of the Weibull fitting
    :return: PSI = Probability of Sample Inclusion. This is the probability that x' is included in the boundary estimated by x_i
    """
    return np.exp(-(((np.abs(dist))/lambda_i)**k_i))

In [ ]:
# Launch the graph
X_evm_train = []
X_evm_test = []
X_evm_data = []

with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess, tf.train.latest_checkpoint("/home/kjakkala/neuralwave/data/weights/central_loss/"))
    
    for step in range(X_test.shape[0]):
        X_evm_test.append(sess.run([embeddings], feed_dict={X: np.expand_dims(X_test[step], axis=0), is_training: False}))
        
    for step in range(X_train.shape[0]):
        X_evm_train.append(sess.run([embeddings], feed_dict={X: np.expand_dims(X_train[step], axis=0), is_training: False}))

    for step in range(X_data.shape[0]):
        X_evm_data.append(sess.run([embeddings], feed_dict={X: np.expand_dims(X_data[step], axis=0), is_training: False}))

    centers_final = sess.run([centers])

X_evm_test = np.squeeze(X_evm_test)
X_evm_train = np.squeeze(X_evm_train)
X_evm_data = np.squeeze(X_evm_data)
centers_final = np.squeeze(centers_final)
print(X_evm_test.shape, X_evm_train.shape, X_evm_data.shape, centers_final.shape)

In [ ]:
tow = 50
weibull_models = []
for i in range(num_classes):
    train_ind = np.squeeze(np.where(y_train_n == i))
    D = pairwise_distances(X_evm_train[train_ind], X_evm_train, metric="euclidean", n_jobs=1)
    for j in range(D.shape[0]):
        D_tmp = np.sort(D[j])
        D_tmp = D_tmp[np.where(D_tmp>0)][:tow]
        weibull_models.append(fit(D_tmp, iters=100, eps=1e-6))
weibull_models = np.array(weibull_models)
weibull_models.shape

In [ ]:
def get_evm(data):
    D = pairwise_distances(X_evm_train, data, metric="euclidean", n_jobs=1)
    preds = np.zeros_like(D)
    for i in range(X_evm_train.shape[0]):
        for j in range(data.shape[0]):
            preds[i, j] = psi_i_dist(D[i, j], weibull_models[i][0], weibull_models[i][1])
    return preds

In [ ]:
evm = get_evm(X_evm_test)
np.mean(np.equal(y_train_n[np.argmax(evm, axis=0)], y_test_n))*100

In [ ]:
def get_preds(data):
    class_thresholds = []
    for i in range(num_classes):
        class_thresholds.append(np.max(np.min(pairwise_distances(X_evm_train[np.where(y_train_n == i)], centers_final), axis=1)))
    class_thresholds = np.array(class_thresholds)
    class_thresholds += class_thresholds*2.4
    
    distance = np.min(pairwise_distances(data, centers_final), axis=1)
    class_preds = np.argmin(pairwise_distances(data, centers_final), axis=1)

    for i in range(data.shape[0]):
        if (distance[i] > class_thresholds[class_preds[i]]):
            class_preds[i] = -1
            
    return class_preds, distance

print(np.mean(np.equal(get_preds(X_evm_train)[0], y_train_n))*100)
print(np.mean(np.equal(get_preds(X_evm_test)[0], y_test_n))*100)
print(np.mean(np.equal(get_preds(X_evm_data)[0], y_data))*100)